In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import datajoint as dj
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
sns.set_style('ticks', rc={'image.cmap': 'bwr'})

import os
import sys
import inspect

p = !pwd
p = os.path.dirname(os.path.dirname(p[0]))
if p not in sys.path:
    sys.path.append(p)

In [ ]:
from cnn_sys_ident.mesonet.data import MultiDataset
from cnn_sys_ident.mesonet.parameters import Core, Readout, Model, RegPath, Fit
from cnn_sys_ident.mesonet import MODELS

In [ ]:
data_key = {'data_hash': 'cfcd208495d565ef66e7dff9f98764da'}

# Model comparison: arbitrary feature weights

In [ ]:
model_rel = MODELS['HermiteSparse'] \
    & 'positive_feature_weights=False AND shared_biases=False'

In [ ]:
def get_stats(model_rel, num_filters=None):
    if num_filters is None:
        num_filters = [8, 12, 16, 20, 24, 28, 32, 40, 48]
    test_corr, val_loss, num_filt = [], [], []
    for n in num_filters:
        filters_key = {'num_filters_2': n}
        rel = Fit() * model_rel & data_key & filters_key
        if len(rel):
            l, r = rel.fetch('val_loss', 'test_corr', order_by='val_loss', limit=5)
            val_loss.append(l)
            test_corr.append(r)
            num_filt.append(n)
            print('{}: {} (SD: {:.2f}), {} (SD: {:.4f})'.format(n, l, l.std(), r, r.std()))

    val_loss = np.array(val_loss)
    test_corr = np.array(test_corr)
    num_filt = np.array(num_filt)
    return val_loss, test_corr, num_filt

In [ ]:
val_loss, test_corr, num_filt = get_stats(model_rel)

In [ ]:
plt.figure(figsize=(4, 3))
plt.plot(num_filt, test_corr, '.k', ms=5)
plt.plot(num_filt, test_corr[:,0], 'o-k')
plt.plot(16, test_corr[num_filt==16,0], 'or', ms=10)
plt.ylim([0.4, 0.5])
plt.xlim([num_filt[0]-1, num_filt[-1]+1])
plt.xticks(num_filt[[0, 2, 4, 6, 7, 8]])
plt.xlabel('Number of feature maps')
plt.ylabel('Correlation on test set')
sns.despine(trim=True, offset=5)
plt.show()

plt.figure(figsize=(4, 3))
plt.plot(num_filt, val_loss, '.k', ms=5)
plt.plot(num_filt, val_loss[:,0], 'ok')
plt.plot(16, val_loss[num_filt==16,0], 'or', ms=10)
plt.ylim([3600, 3700])
plt.xlim([num_filt[0]-1, num_filt[-1]+1])
plt.xticks(num_filt[[0, 2, 4, 6, 7, 8]])
plt.xlabel('Number of feature maps')
plt.ylabel('Loss on validation set')
sns.despine(trim=True, offset=5)
plt.savefig('figures/model_comp.eps', format='eps')
plt.show()

# Disrtribution of correlations across neurons

In [ ]:
import scipy.stats
model_rel = MODELS['HermiteSparse'] \
    & 'positive_feature_weights=False AND shared_biases=False AND num_filters_2=16'

In [ ]:
key = (Fit() * model_rel).fetch(dj.key, order_by='val_loss')[1]
model = Fit().load_model(key)

In [ ]:
data = model.base.data
inputs, responses = data.test()
feed_dict = {model.base.inputs: inputs, model.base.is_training: False}
predictions = model.base.evaluate(model.predictions, feed_dict=feed_dict)

In [ ]:
rho = [scipy.stats.pearsonr(p, r)[0] for p, r in zip(predictions.T, responses.T)]

In [ ]:
plt.figure(figsize=(3, 2))
sns.distplot(rho, bins=np.linspace(-0.2, 1, 50), kde=False)
plt.xlabel('Test correlation')
plt.ylabel('Number of neurons')
plt.xlim([-0.2, 1])
plt.ylim([0, 400])
sns.despine(offset=5)
plt.tight_layout()
plt.savefig('figures/test_correlations_distr.eps', format='eps')

# Control: positive feature weights

In [ ]:
model_rel_p = MODELS['HermiteSparse'] \
    & 'positive_feature_weights=True AND shared_biases=False'
val_loss_p, test_corr_p, num_filt_p = get_stats(model_rel_p)

In [ ]:
plt.figure(figsize=(4, 3))
plt.plot(num_filt, test_corr, '.k', ms=5)
plt.plot(num_filt, test_corr[:,0], 'o-k')
plt.plot(num_filt_p, test_corr_p[:,0], 'o-g')
plt.plot(num_filt_p, test_corr_p[:,:3], '.g', ms=5)
plt.plot(16, test_corr[num_filt==16,0], 'or', ms=10)
plt.ylim([0.4, 0.5])
plt.xlim([num_filt[0]-1, num_filt[-1]+1])
plt.xticks(num_filt[[0, 2, 4, 5, 6, 7]])
plt.xlabel('Number of feature maps')
plt.ylabel('Correlation on test set')
sns.despine(trim=True, offset=5)
plt.show()

plt.figure(figsize=(4, 3))
plt.plot(num_filt, val_loss, '.k', ms=5)
plt.plot(num_filt, val_loss[:,0], 'ok')
plt.plot(num_filt_p, val_loss_p[:,0], 'og')
plt.plot(num_filt_p, val_loss_p[:,:3], '.g', ms=5)
plt.plot(16, val_loss[num_filt==16,0], 'or', ms=10)
plt.ylim([3600, 3800])
plt.xlim([num_filt[0]-1, num_filt[-1]+1])
plt.xticks(num_filt[[0, 2, 4, 6, 7, 8]])
plt.xlabel('Number of feature maps')
plt.ylabel('Loss on validation set')
sns.despine(trim=True, offset=5)
plt.show()

# Control: shared biases

In [ ]:
model_rel_b = MODELS['HermiteSparse'] \
    & 'positive_feature_weights=False AND shared_biases=True'
val_loss_b, test_corr_b, num_filt_b = get_stats(model_rel_b, num_filters=[16, 32, 48])

In [ ]:
plt.figure(figsize=(4, 3))
plt.plot(num_filt, test_corr, '.k', ms=5)
plt.plot(num_filt, test_corr[:,0], 'o-k')
plt.plot(num_filt_b, test_corr_b[:,0], 'o-b')
plt.plot(num_filt_b, test_corr_b[:,:3], '.b', ms=5)
plt.plot(16, test_corr[num_filt==16,0], 'or', ms=10)
plt.ylim([0.4, 0.5])
plt.xlim([num_filt[0]-1, num_filt[-1]+1])
plt.xticks(num_filt[[0, 2, 4, 5, 6, 7]])
plt.xlabel('Number of feature maps')
plt.ylabel('Correlation on test set')
sns.despine(trim=True, offset=5)
plt.show()

plt.figure(figsize=(4, 3))
plt.plot(num_filt, val_loss, '.k', ms=5)
plt.plot(num_filt, val_loss[:,0], 'ok')
plt.plot(num_filt_b, val_loss_b[:,0], 'o-b')
plt.plot(num_filt_b, val_loss_b[:,:3], '.b', ms=5)
plt.plot(16, val_loss[num_filt==16,0], 'or', ms=10)
plt.ylim([3600, 3700])
plt.xlim([num_filt[0]-1, num_filt[-1]+1])
plt.xticks(num_filt[[0, 2, 4, 6, 7, 8]])
plt.xlabel('Number of feature maps')
plt.ylabel('Loss on validation set')
sns.despine(trim=True, offset=5)
plt.show()